In [1]:
%matplotlib notebook

In [2]:
import pathlib
import numpy as np
import matplotlib.dates
import matplotlib.pyplot as plt
import sunpy.net.attrs
import sunpy.net.dataretriever.attrs.goes
import sunpy.timeseries
import kgpy.observatories.iris
import iris_planning.week_2021_12_13.hop_307.ar12907.data

In [3]:
result = sunpy.net.Fido.search(
    sunpy.net.attrs.Time(
        start='2021-12-14 04:00',
        end='2021-12-21 04:00'
    ),
    sunpy.net.attrs.Instrument('XRS'),
    sunpy.net.dataretriever.attrs.goes.SatelliteNumber(16),
)

In [4]:
files = sunpy.net.Fido.fetch(result)

Files Downloaded:   0%|          | 0/8 [00:00<?, ?file/s]

In [5]:
timeseries = sunpy.timeseries.TimeSeries(files, concatenate=True)

In [6]:
fig, ax = plt.subplots(figsize=(12, 4))
timeseries.plot()
formatter = matplotlib.dates.DateFormatter("%m-%d %H:%M")
ax.xaxis.set_major_formatter(formatter)
ax.set_xlabel('date')
fig.savefig('goes.png', dpi=600)

<IPython.core.display.Javascript object>

In [7]:
folder_input = pathlib.Path(iris_planning.week_2021_12_13.hop_307.ar12907.data.__file__).parent

archive_sequence = sorted(folder_input.glob('*raster.tar.gz'))

cube = kgpy.observatories.iris.spectrograph.CubeList(
    [kgpy.observatories.iris.spectrograph.Cube.from_archive(archive) for archive in archive_sequence]
)
shape_w_min = np.array([c.shape[~0] for c in cube]).min()
for c in cube:
    c.intensity = c.intensity[..., :shape_w_min]
    # c.intensity_uncertainty = c.intensity_uncertainty[..., :shape_w_min]

cube = cube.to_cube()

In [8]:
intensity = cube.intensity

In [9]:
colors = cube.colors

c:\users\byrdie\appdata\local\programs\python\python38\lib\site-packages\astropy\units\quantity.py:486: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
c:\users\byrdie\appdata\local\programs\python\python38\lib\site-packages\astropy\units\quantity.py:486: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
c:\users\byrdie\appdata\local\programs\python\python38\lib\site-packages\astropy\units\quantity.py:486: RuntimeWarning: invalid value encountered in sqrt
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [10]:
fig, ax = plt.subplots(figsize=(16, 9), constrained_layout=True)
ax.set_facecolor('black')
ax.imshow(
    X=np.swapaxes(colors[0, 0], 0, 1),
    origin='lower',
)

<IPython.core.display.Javascript object>

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [11]:
fig2, ax2 = plt.subplots(nrows=4, ncols=2, constrained_layout=True)
ax2[0, 0].plot(intensity[0, 0, 81, 285])
ax2[0, 1].set_facecolor(np.clip(colors[0, 0, 81, 285].value, 0, 1))
ax2[1, 0].plot(intensity[0, 0, 57, 273])
ax2[1, 1].set_facecolor(np.clip(colors[0, 0, 57, 273].value, 0, 1))
ax2[2, 0].plot(intensity[0, 0, 39, 270])
ax2[2, 1].set_facecolor(np.clip(colors[0, 0, 39, 270].value, 0, 1))
ax2[3, 0].plot(intensity[0, 0, 60, 341])
ax2[3, 1].set_facecolor(np.clip(colors[0, 0, 60, 341].value, 0, 1))
fig2.savefig('colors.png', dpi=600)

<IPython.core.display.Javascript object>

In [12]:
colors[0, 0, 81, 285].value

array([0.12889261, 0.54031001, 1.        , 1.32842569])

In [19]:
import scipy.ndimage
ts_array = timeseries.to_dataframe()['xrsa'].to_numpy()
ts_array = np.log10(ts_array)
ts_array = np.nan_to_num(ts_array, nan=np.nanmedian(ts_array))
# ts_array = 2 * (ts_array - np.nanmin(ts_array)) / (np.nanmax(ts_array) - np.nanmin(ts_array)) - 1
kernel_radius = 10000
kernel_diameter = 2 * kernel_radius + 1
# ts_array_mean = np.convolve(ts_array, np.ones(kernel_diameter)/kernel_diameter, mode='valid')
ts_array_avg = scipy.ndimage.median_filter(ts_array, kernel_diameter)

<ipython-input-19-d93d39ab2a6b>:3: RuntimeWarning: invalid value encountered in log10
  ts_array = np.log10(ts_array)


In [20]:
ts_array_norm = ts_array - ts_array_avg
# ts_array_norm = ts_array_norm / (-ts_array_norm.min())
# ts_array_norm = np.sign(ts_array_norm) * np.sqrt(np.abs(ts_array_norm))
# ts_array_norm = np.clip(ts_array_norm, -1, 1)

In [21]:
ts_array.min()

-12.289516

In [22]:
plt.figure()
plt.plot(ts_array)
plt.plot(ts_array_avg)

<IPython.core.display.Javascript object>

In [23]:
plt.figure()
plt.plot(ts_array_norm)

<IPython.core.display.Javascript object>

In [24]:
import IPython.display
IPython.display.Audio(ts_array_norm, rate=20000)